In [6]:
import pandas as pd
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool
from processing_utils import process_file

In [2]:
res = pd.read_csv("/Volumes/T9/FOXES_Data/paper_res/vit/vit_predictions.csv")

In [3]:
res

,timestamp,predictions,groundtruth
0,2023-08-01T00:00:00,0.000003,0.000003
1,2023-08-01T00:01:00,0.000003,0.000003
2,2023-08-01T00:03:00,0.000003,0.000003
3,2023-08-01T00:04:00,0.000003,0.000003
4,2023-08-01T00:05:00,0.000003,0.000003
...,...,...,...
100192,2025-09-26T20:17:00,0.000004,0.000004
100193,2025-09-26T20:18:00,0.000004,0.000004
100194,2025-09-26T20:19:00,0.000003,0.000004
100195,2025-09-26T20:20:00,0.000003,0.000003


In [14]:
#Develop a linear regression model for a baseline!
training_data_path = "/Volumes/T9/FOXES_Data/AIA/train"



# Note: In some environments, you might need: 
# from notebook_tests.processing_utils import process_file

files = [f for f in os.listdir(training_data_path) if f.endswith('.npy')]
file_infos = [(training_data_path, f) for f in files]

# Use multiprocessing to speed up data loading
with Pool(processes=os.cpu_count()) as pool:
    sums = list(tqdm(pool.imap(process_file, file_infos), total=len(files)))

# Convert to dataframe for easier handling
wavelengths = ['94', '131', '171', '193', '211', '304', '335']
results_df = pd.DataFrame(sums, columns=[f'sum_flux_{w}' for w in wavelengths])
results_df['filename'] = files
results_df['timestamp'] = pd.to_datetime(results_df['filename'].str.replace('.npy', '', regex=False))
results_df = results_df.sort_values('timestamp')




100%|██████████| 84165/84165 [12:36<00:00, 111.30it/s]


In [ ]:
print(results_df)
results_df.to_csv("/Volumes/T9/FOXES_Data/train_flux_sums.csv", index=False)
# %%
aia = pd.read_csv("/Volumes/T9/FOXES_Data/train_flux_sums.csv")
# %%
aia
# %%
# Load SXR data



In [7]:
sxr_training_data_path = "/Volumes/T9/FOXES_Data/SXR/train"
sxr_files = [f for f in os.listdir(sxr_training_data_path) if f.endswith('.npy')]
sxr_file_infos = [(sxr_training_data_path, f) for f in sxr_files]

with Pool(processes=os.cpu_count()) as pool:
    sxr_fluxes = list(tqdm(pool.imap(process_file, sxr_file_infos), total=len(sxr_files)))

sxr_df = pd.DataFrame(sxr_fluxes, columns=['sxr_flux'])
sxr_df['filename'] = sxr_files
sxr_df['timestamp'] = pd.to_datetime(sxr_df['filename'].str.replace('.npy', '', regex=False))
sxr_df = sxr_df.sort_values('timestamp')

# %%
sxr_df
# %%
# Merge AIA and SXR data
# Convert aia timestamp to datetime if not already (it's read from csv, so it's likely string)
aia['timestamp'] = pd.to_datetime(aia['timestamp'])
merged_df = pd.merge(aia, sxr_df[['timestamp', 'sxr_flux']], on='timestamp', how='inner')
# %%
merged_df

100%|██████████| 77809/77809 [05:37<00:00, 230.48it/s] 


In [ ]:
sxr_norms = np.load('/Volumes/T9/FOXES_Data/SXR/normalized_sxr.npy')

In [ ]:
sxr_norms

In [ ]:
def normalize_sxr(unnormalized_values, sxr_norm):
    """Convert from unnormalized to normalized space"""
    log_values = np.log10(unnormalized_values + 1e-8)
    normalized = (log_values - float(sxr_norm[0].item())) / float(sxr_norm[1].item())
    return normalized

In [ ]:
def unnormalize_sxr(normalized_values, sxr_norm):
    return 10 ** (normalized_values * float(sxr_norm[1].item()) + float(sxr_norm[0].item())) - 1e-8

In [ ]:
merged_df['sxr_flux_normalized'] = normalize_sxr(merged_df['sxr_flux'].values, sxr_norms)
merged_df.to_csv("/Volumes/T9/FOXES_Data/train_data_combined.csv", index=False)

In [ ]:
merged_df

In [24]:
val_training_path = "/Volumes/T9/FOXES_Data/AIA/val"



# Note: In some environments, you might need:
# from notebook_tests.processing_utils import process_file

files = [f for f in os.listdir(val_training_path) if f.endswith('.npy')]
file_infos = [(val_training_path, f) for f in files]

# Use multiprocessing to speed up data loading
with Pool(processes=os.cpu_count()) as pool:
    sums = list(tqdm(pool.imap(process_file, file_infos), total=len(files)))

# Convert to dataframe for easier handling
wavelengths = ['94', '131', '171', '193', '211', '304', '335']
results_df = pd.DataFrame(sums, columns=[f'sum_flux_{w}' for w in wavelengths])
results_df['filename'] = files
results_df['timestamp'] = pd.to_datetime(results_df['filename'].str.replace('.npy', '', regex=False))
val_results_df = results_df.sort_values('timestamp')

100%|██████████| 16317/16317 [02:15<00:00, 120.65it/s]


In [26]:
val_results_df.to_csv("/Volumes/T9/FOXES_Data/val_flux_sums.csv", index=False)
aia = pd.read_csv("/Volumes/T9/FOXES_Data/val_flux_sums.csv")

In [27]:
sxr_training_data_path = "/Volumes/T9/FOXES_Data/SXR/val"
sxr_files = [f for f in os.listdir(sxr_training_data_path) if f.endswith('.npy')]
sxr_file_infos = [(sxr_training_data_path, f) for f in sxr_files]

with Pool(processes=os.cpu_count()) as pool:
    sxr_fluxes = list(tqdm(pool.imap(process_file, sxr_file_infos), total=len(sxr_files)))

sxr_df = pd.DataFrame(sxr_fluxes, columns=['sxr_flux'])
sxr_df['filename'] = sxr_files
sxr_df['timestamp'] = pd.to_datetime(sxr_df['filename'].str.replace('.npy', '', regex=False))
sxr_df = sxr_df.sort_values('timestamp')

# %%
sxr_df
# %%
# Merge AIA and SXR data
# Convert aia timestamp to datetime if not already (it's read from csv, so it's likely string)
aia['timestamp'] = pd.to_datetime(aia['timestamp'])
merged_df_val = pd.merge(aia, sxr_df[['timestamp', 'sxr_flux']], on='timestamp', how='inner')
# %%
merged_df_val

100%|██████████| 22718/22718 [01:04<00:00, 352.67it/s]


,sum_flux_94,sum_flux_131,sum_flux_171,sum_flux_193,sum_flux_211,sum_flux_304,sum_flux_335,filename,timestamp,sxr_flux
0,-200676.10,-197662.77,-104301.250,-70421.400,-98666.580,-124224.820,-156857.17,2023-03-01T00:41:00.npy,2023-03-01 00:41:00,0.000002
1,-200782.50,-197657.44,-104293.190,-70402.910,-98640.880,-124406.090,-156618.22,2023-03-01T00:42:00.npy,2023-03-01 00:42:00,0.000002
2,-200727.90,-197653.10,-104272.850,-70396.266,-98656.960,-124446.970,-156893.97,2023-03-01T00:43:00.npy,2023-03-01 00:43:00,0.000002
3,-200796.84,-197660.70,-104259.930,-70401.690,-98654.970,-124449.766,-156981.73,2023-03-01T00:44:00.npy,2023-03-01 00:44:00,0.000002
4,-200810.61,-197684.56,-104275.740,-70392.230,-98631.305,-124564.030,-156861.61,2023-03-01T00:45:00.npy,2023-03-01 00:45:00,0.000002
...,...,...,...,...,...,...,...,...,...,...
16312,-198742.75,-196629.39,-107412.484,-67689.920,-92760.555,-123725.180,-149980.58,2023-07-30T23:55:00.npy,2023-07-30 23:55:00,0.000001
16313,-198719.10,-196587.69,-107403.330,-67713.780,-92780.375,-123720.240,-149841.75,2023-07-30T23:56:00.npy,2023-07-30 23:56:00,0.000001
16314,-198798.88,-196677.50,-107400.810,-67713.790,-92773.170,-123854.510,-149851.62,2023-07-30T23:57:00.npy,2023-07-30 23:57:00,0.000001
16315,-198725.50,-196521.72,-107419.720,-67721.570,-92776.766,-123768.940,-150048.25,2023-07-30T23:58:00.npy,2023-07-30 23:58:00,0.000001


In [28]:
merged_df_val['sxr_flux_normalized'] = normalize_sxr(merged_df_val['sxr_flux'].values, sxr_norms)
merged_df_val.to_csv("/Volumes/T9/FOXES_Data/val_data_combined.csv", index=False)

In [29]:
test_training_path = "/Volumes/T9/FOXES_Data/AIA/test"



# Note: In some environments, you might need:
# from notebook_tests.processing_utils import process_file

files = [f for f in os.listdir(test_training_path) if f.endswith('.npy')]
file_infos = [(test_training_path, f) for f in files]

# Use multiprocessing to speed up data loading
with Pool(processes=os.cpu_count()) as pool:
    sums = list(tqdm(pool.imap(process_file, file_infos), total=len(files)))

# Convert to dataframe for easier handling
wavelengths = ['94', '131', '171', '193', '211', '304', '335']
results_df = pd.DataFrame(sums, columns=[f'sum_flux_{w}' for w in wavelengths])
results_df['filename'] = files
results_df['timestamp'] = pd.to_datetime(results_df['filename'].str.replace('.npy', '', regex=False))
test_results_df = results_df.sort_values('timestamp')

100%|██████████| 100197/100197 [15:54<00:00, 104.97it/s]


In [30]:
test_results_df.to_csv("/Volumes/T9/FOXES_Data/test_flux_sums.csv", index=False)
aia = pd.read_csv("/Volumes/T9/FOXES_Data/test_flux_sums.csv")

In [31]:
sxr_training_data_path = "/Volumes/T9/FOXES_Data/SXR/test"
sxr_files = [f for f in os.listdir(sxr_training_data_path) if f.endswith('.npy')]
sxr_file_infos = [(sxr_training_data_path, f) for f in sxr_files]

with Pool(processes=os.cpu_count()) as pool:
    sxr_fluxes = list(tqdm(pool.imap(process_file, sxr_file_infos), total=len(sxr_files)))

sxr_df = pd.DataFrame(sxr_fluxes, columns=['sxr_flux'])
sxr_df['filename'] = sxr_files
sxr_df['timestamp'] = pd.to_datetime(sxr_df['filename'].str.replace('.npy', '', regex=False))
sxr_df = sxr_df.sort_values('timestamp')

# %%
sxr_df
# %%
# Merge AIA and SXR data
# Convert aia timestamp to datetime if not already (it's read from csv, so it's likely string)
aia['timestamp'] = pd.to_datetime(aia['timestamp'])
merged_df_test = pd.merge(aia, sxr_df[['timestamp', 'sxr_flux']], on='timestamp', how='inner')
# %%
merged_df_test

100%|██████████| 100197/100197 [13:16<00:00, 125.81it/s]


,sum_flux_94,sum_flux_131,sum_flux_171,sum_flux_193,sum_flux_211,sum_flux_304,sum_flux_335,filename,timestamp,sxr_flux
0,-196843.72,-196055.12,-106819.914,-68804.000,-92321.470,-122797.240,-147060.34,2023-08-01T00:00:00.npy,2023-08-01 00:00:00,0.000003
1,-196831.34,-196110.30,-108905.910,-68786.055,-92310.360,-122770.040,-147090.06,2023-08-01T00:01:00.npy,2023-08-01 00:01:00,0.000003
2,-196688.38,-196193.31,-108975.700,-68785.766,-92311.945,-122728.190,-147227.08,2023-08-01T00:03:00.npy,2023-08-01 00:03:00,0.000003
3,-196823.53,-196036.02,-109010.380,-68791.820,-92297.500,-122846.625,-147089.60,2023-08-01T00:04:00.npy,2023-08-01 00:04:00,0.000003
4,-196832.25,-196109.80,-108999.086,-68813.810,-92325.640,-122816.240,-147055.19,2023-08-01T00:05:00.npy,2023-08-01 00:05:00,0.000003
...,...,...,...,...,...,...,...,...,...,...
100192,-194550.94,-201906.84,-116466.310,-87035.090,-111005.910,-134399.690,-151818.72,2025-09-26T20:17:00.npy,2025-09-26 20:17:00,0.000004
100193,-194584.00,-201919.28,-116529.310,-87337.625,-111037.630,-134419.750,-152003.50,2025-09-26T20:18:00.npy,2025-09-26 20:18:00,0.000004
100194,-194670.88,-201917.78,-116512.400,-87066.450,-111055.766,-134361.980,-151734.81,2025-09-26T20:19:00.npy,2025-09-26 20:19:00,0.000004
100195,-194634.40,-201907.12,-116535.984,-87106.110,-111052.000,-134395.970,-151819.88,2025-09-26T20:20:00.npy,2025-09-26 20:20:00,0.000003


In [32]:
merged_df_test['sxr_flux_normalized'] = normalize_sxr(merged_df_test['sxr_flux'].values, sxr_norms)
merged_df_test.to_csv("/Volumes/T9/FOXES_Data/test_data_combined.csv", index=False)